Code copied by Klaus from https://cp4d-cpd-cp4d.emer2gent.ml/analytics/notebooks/v2/937c9ce5-dbec-43a5-8756-27db4eb087da/view?access_token=6694141e3512c588ca4a53a32b6e05624435f89bb540be8d7d37901b2eafb63e

In [ ]:
def forecast(df,x_column_name="datetime",y_column_name="new_cases_7_per_100000",look_back=7,look_forward=14):
    y = df[[y_column_name]].values[-look_back:].reshape(-1,1)
    X = [[float(i)] for i in range(0,look_back)]
    x = [[float(i)] for i in range(0,look_back+look_forward)]
    y_linear = np.squeeze(LinearRegression().fit(X, y).predict(x))[look_back:]
    y_linear[y_linear<0] = 0
    y_linear = np.insert(y_linear,0,df[y_column_name].values[-1])
    
    y = df[y_column_name].values[-look_back:]
    mf_AR = AR(y).fit(trend='nc')
    y_ar = mf_AR.predict(start=len(y),end=len(y)+look_forward-1,dynamic=True)
    y_ar[y_ar<0] = 0
    y_ar = np.insert(y_ar,0,df[y_column_name].values[-1])
    
    mf_SARIMAX = SARIMAX(y).fit()
    y_sarimax = mf_SARIMAX.predict(start=len(y),end=len(y)+look_forward-1,dynamic=True)
    y_sarimax[y_sarimax<0] = 0
    y_sarimax = np.insert(y_sarimax,0,df[y_column_name].values[-1])
    
    real_x = [df[x_column_name].values.min()+pd.Timedelta(days=i) for i in range(look_back-1,look_back+look_forward)]
    
    y_min = []
    y_max = []
    for i in range(len(y_linear)):
        y_min.append(min(y_linear[i],y_ar[i],y_sarimax[i]))
        y_max.append(max(y_linear[i],y_ar[i],y_sarimax[i]))
        
    return mf_AR,mf_SARIMAX,pd.DataFrame({x_column_name:real_x,
                         y_column_name+"_linear":y_linear,
                         y_column_name+"_ar":y_ar,
                         y_column_name+"_sarimax":y_sarimax,
                         y_column_name+"_max":y_max,
                         y_column_name+"_min":y_min})